<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run Losses

Goal : have a table that compares the different losses on MNIST.

Data : MNIST

Models : AttnNPFXL and ConvNPFXL for CNPFLoss, NLLLloss, ISLoss, ElboLoss

Runs : 5

In [1]:
import os

os.chdir("..")

import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [2]:
args = get_exp_args("exp_final3_losses", is_load=False)
len(args)

30

In [4]:
executor=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24,  
                           cpus_per_task=10, 
                           mem='32GB',
                           partition="priority",
                           comment="neurips",
                          )

In [5]:
jobs = executor.map_array(Run(), args)

In [6]:
############################################################

In [17]:
jobs

[SlurmJob<job_id=27110876_0, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_1, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_3, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_4, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_5, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_6, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_7, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_8, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_9, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_10, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_11, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_12, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_13, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_14, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_15, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27110876_16, task

In [15]:
import numpy as np
from scipy.stats import sem
sem(np.array([4193.4430, 4219, 4229.1555, 4217.1758, 4173.3125]))

10.137444560371184

In [8]:
for j in jobs:
    print("--------------------------------")
    print(j.stdout())

--------------------------------
submitit INFO (2020-05-31 12:25:35,456) - Starting with JobEnvironment(job_id=27079445_0, hostname=learnfair0526, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-05-31 12:25:35,456) - Loading pickle: /private/home/yannd/projects/NPF/logs/27079445_0/27079445_0_submitted.pkl

--- Training mnist/ConvNPFXL_NllLNPF/run_0 ---

  epoch    train_loss    valid_loss    cp       dur
-------  ------------  ------------  ----  --------
      1     -837.3610    -1340.7969     +  858.3704
      2    -1323.1655    -1541.7803     +  857.9864
      3    -1565.9752    -1518.8323        858.1118
      4    -1662.8694    -1711.5384     +  858.9704
      5    -1730.3656    -1833.1489     +  858.1543
      6    -1782.3099    -1919.1480     +  858.2080
      7    -1799.9358    -1863.3516        858.6080
      8    -1867.4006    -1924.1199     +  858.5089
      9    -1888.2093    -1931.8876     +  859.1300
     10    -1905.6517    -1788.6149        859.8754
  

In [10]:
for j in jobs:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
submitit ERROR (2020-05-30 03:20:14,182) - Submitted job triggered an exception
ERROR:submitit:Submitted job triggered an exception
Traceback (most recent call last):
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/_submit.py", line 6, in <module>
    submitit_main()
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 186, in submitit_main
    process_job(args.folder)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 179, in process_job
    raise error
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submi

In [7]:
for job in jobs:
    job.cancel()